In [2]:
import uuid
# project_id = str(uuid.uuid4())
run_id = str(uuid.uuid4())
print(run_id)

3dd51bfe-b4ee-4229-aa1f-8cf8170c97ef


In [3]:
# payload-1 (classification Airline Delay dataset)
exp_data = '''{{
"source":"EXP", 
"project_id":"0e0fb803-22db-4d88-9f2f-f6f75b6abcf0", 
"id":"7bbb5061-54d4-4862-8d47-7fbee388a4d1", 
"run_id":"{0}", 
"exp_name": "Final_recipe", 
"algorithm_type":"classification", 
"algo_details": {{"snowflake.ml.modeling.xgboost.XGBClassifier": null}}, 
"dataset": "AIRLINE_DEP_DELAY_10K", 
"target_column": "DEP_DEL15"}}'''.format(run_id)

In [4]:
exp_data

'{\n"source":"EXP", \n"project_id":"0e0fb803-22db-4d88-9f2f-f6f75b6abcf0", \n"id":"7bbb5061-54d4-4862-8d47-7fbee388a4d1", \n"run_id":"3dd51bfe-b4ee-4229-aa1f-8cf8170c97ef", \n"exp_name": "Final_recipe", \n"algorithm_type":"classification", \n"algo_details": {"snowflake.ml.modeling.xgboost.XGBClassifier": null}, \n"dataset": "AIRLINE_DEP_DELAY_10K", \n"target_column": "DEP_DEL15"}'

In [5]:
from fosforio.manager import get_conn_details_from_ds_name
from snowflake.snowpark.session import Session
import json

Connection manager service url initialised to http://fdc-project-manager:80/project-manager
If you need to update its value then update the variable CONNECTION_MANAGER_BASE_URL in os env.


In [6]:
# def create_stage(session, stage_name="demo"):
#     try:
#         session.sql(f"create or replace stage {stage_name}").collect()
#         return f"@{stage_name}"
#     except Exception as ex:
#         print("Error while creating snowflake session", ex)
#         raise ex

        
def get_session(dataset, project_id):
    """
    Method creates snowflake session object.
    :return:
    """
    try:
        conn = get_conn_details_from_ds_name(dataset, project_id)
        region=conn["params"]["READER"]["region"] if conn["params"]["READER"]["cloudPlatform"] is None \
                    else conn["params"]["READER"]["region"]+"."+conn["params"]["READER"]["cloudPlatform"]
        account = conn['params']['READER']['accountId'] if region is None \
                    else conn['params']['READER']['accountId']+"."+region
        CONNECTION_PARAMETERS = {
            "account": account,
            "user":conn['params']['READER']['user'],
            "password": conn['params']['READER']['password'],
            "role": conn['params']['READER']['role'],
            "database": conn['params']['READER']['database'],
            "warehouse": conn['params']['READER']['wareHouse'],
            "schema": conn['params']['READER']['schema']
        }
        return Session.builder.configs(CONNECTION_PARAMETERS).create()
    except Exception as ex:
        print("Error while creating snowflake session", ex)
        raise ex

In [7]:
# Stored Procedure 
def run_experiment(session: Session, exp_data: str) -> list:
    # Imports
    try:
        from fosforio.manager import get_conn_details_from_ds_name
        exp_details=json.loads(exp_data)
        conn = get_conn_details_from_ds_name(exp_details.get("dataset"),exp_details.get("project_id"))
        return [{"connection details": conn}]
    except Exception as ex:
        print("Exception Occured in run experiment")
        return str(ex).split('?')

In [17]:
%%time
# Initilization
# logging.basicConfig(stream=sys.stdout, level=logging.INFO)
print("Creating Snowflake Session object...")
exp_details=json.loads(exp_data)
session = get_session(exp_details.get("dataset"),exp_details.get("project_id"))
stage = create_stage(session)
print("Session has been created !")

# snowflake_environment = session.sql('SELECT current_user(), current_version()').collect()
# Current Environment Details
# print('\nConnection Established with the following parameters:')
# print('User                        : {}'.format(snowflake_environment[0][0]))
# print('Role                        : {}'.format(session.get_current_role()))
# print('Database                    : {}'.format(session.get_current_database()))
# print('Schema                      : {}'.format(session.get_current_schema()))
# print('Warehouse                   : {}'.format(session.get_current_warehouse()))

print("Creating stored procedure...")
session.custom_package_usage_config['enabled'] = True
session.custom_package_usage_config['force_push'] = True

session.sproc.register(func=run_experiment,
                       name="run_experiment",
                       packages=["snowflake-snowpark-python", "snowflake-ml-python","fosforio"],
                       isPermanant=False,
                       stage_location=stage,
                       replace=True)
print("Stored procedure has been created successfully!")

# tagging session
print("Setting tag to session object: tag= ", run_id)
session.query_tag=run_id

print("Executing Procedure")
procedure_response = session.call("run_experiment", exp_data)
# procedure_response = run_experiment(session, exp_data)
print("Stored Procedure Executed Successfully !")
print(procedure_response)

#Log in mlflow
print("Logging in snowpark registry completed !")

Creating Snowflake Session object...
Session has been created !
Creating stored procedure...


The version of package 'snowflake-snowpark-python' in the local environment is 1.19.0, which does not fit the criteria for the requirement 'snowflake-snowpark-python'. Your UDF might not work when the package version is different between the server and your local environment.
The version of package 'snowflake-ml-python' in the local environment is 1.5.3, which does not fit the criteria for the requirement 'snowflake-ml-python'. Your UDF might not work when the package version is different between the server and your local environment.
The following packages are not available in Snowflake: ['fosforio'].
If you are adding package(s) unavailable in Snowflake, it is highly recommended that you include the 'cache_path' configuration parameter in order to reduce latency.
ERROR: snowflake-ml-python 1.5.3 has requirement numpy<2,>=1.23, but you'll have numpy 2.0.0 which is incompatible.
ERROR: snowflake-connector-python 3.11.0 has requirement urllib3<2.0.0,>=1.21.1; python_version < "3.10", bu

Stored procedure has been created successfully!
Setting tag to session object: tag=  3dd51bfe-b4ee-4229-aa1f-8cf8170c97ef
Executing Procedure
Stored Procedure Executed Successfully !
[
  "HTTPConnectionPool(host='fdc-project-manager', port=80): Max retries exceeded with url: /project-manager/connections/api/External/v2/external/getConnConfig/AIRLINE_DEP_DELAY_10K/fdcuser/0e0fb803-22db-4d88-9f2f-f6f75b6abcf0 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f444b59bdc0>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))"
]
Logging in mlflow completed !
CPU times: user 854 ms, sys: 172 ms, total: 1.03 s
Wall time: 19.6 s
